##ENCODE SEQUENCE

In [2]:
import pandas as pd
from Bio.SeqIO import parse
from Bio.SeqRecord import SeqRecord
from Bio.Seq import Seq

from sklearn.preprocessing import LabelEncoder

import numpy as np
import re

In [3]:
path="final.csv"

data_df=pd.read_csv(path)

data_df.head(1)

,Accession,Release_Date,Species,Length,Geo_Location,Host,Isolation_Source,Collection_Date,Sequence
0,NC_045512,2020-01-13T00:00:00Z,Severe acute respiratory syndrome-related coro...,29903.0,China,Homo sapiens,NaN,2019-12,ATTAAAGGTTTATACCTTCCCAGGTAACAAACCAACCAACTTTCGA...


function to convert a *DNA sequence* string to a *numpy array*

converts to lower case, changes any non *'acgt'* characters to *'n'*

In [ ]:
def string_to_array(my_string):
    my_string = my_string.lower()
    my_string = re.sub('[^acgt]', 'z', my_string)
    my_array = np.array(list(my_string))
    return my_array

create a label encoder with *'acgtn' alphabet

In [4]:
label_encoder = LabelEncoder()
label_encoder.fit(np.array(['a','c','g','t','z']))

function to encode a DNA sequence string as an *ordinal vector*

returns a *numpy vector* with *a=0.25, c=0.50, g=0.75, t=1.00, n=0.00*

In [ ]:
def ordinal_encoder(my_array):
    
    integer_encoded = label_encoder.transform(my_array)
    float_encoded = integer_encoded.astype(float)
    float_encoded[float_encoded == 0] = 0.25 # A
    float_encoded[float_encoded == 1] = 0.50 # C
    float_encoded[float_encoded == 2] = 0.75 # G
    float_encoded[float_encoded == 3] = 1.00 # T
    float_encoded[float_encoded == 4] = 0.00 # anything else, z
    
    return float_encoded

Creating *final_data* dataframe with *'encoded-sequence'* and *'Geo_location'* columns

In [5]:
data=data_df[["Sequence","Geo_Location"]]

dummy=[]
dum=np.array(dummy)

form={"inp_seq":dum}

seq_df = pd.DataFrame (form, columns = ['inp_seq'])

seq_list=[]

for idx, seq in enumerate(list(data["Sequence"])):
    arr=ordinal_encoder(string_to_array(seq))
    seq_list.append(arr)
    
seq_df["inp_seq"]=seq_list

final_data= data.assign(enc_seq=seq_df)

final_data=final_data[["enc_seq","Geo_Location"]]

final_data.head(5)

,enc_seq,Geo_Location
0,"[0.25, 1.0, 1.0, 0.25, 0.25, 0.25, 0.75, 0.75,...",China
1,"[0.5, 0.25, 0.25, 0.75, 1.0, 0.5, 0.25, 1.0, 0...",China: Nanchang
2,"[0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0.25, 1....",USA: VA
3,"[0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5, 0.75, 0....",USA: VA
4,"[0.5, 0.5, 0.25, 0.25, 0.5, 1.0, 1.0, 1.0, 0.5...",USA: VA
